# GPT-4 Zero-Shot Classification Model

In [1]:
# !pip install openai

In [3]:
import pandas as pd
import numpy as np
import pickle

### 20% for the Social Media Data for GPT Classification

In [4]:
test_dataset = pd.read_csv("path/to/test_data/")

In [5]:
test_dataset.domain.value_counts()

domain
0    3384
1    2793
2    1509
Name: count, dtype: int64

# Define Parameters

### GPT version used for generation

In [6]:
gpt_version = "gpt-4-0125-preview" # change to this for use GPT3 "gpt-3.5-turbo-0125"

### Pricing

In [7]:
pricings = {"gpt-4-0125-preview":(10,30),
            "gpt-3.5-turbo-0125": (0.50, 1.50)}

### Max Tokens

In [8]:
max_tokens = 100

### Moral Values and their Descriptors

In [9]:
moral_values = {"Care": "underlies the virtues of kindness and nurturance, empathy and compassion towards other people",
                "Harm": "which represents the opposite of care, characterised by violent actions and intentions",
                "Fairness": " which underlies the virtues of honesty, fair treatment, justice, and dependability.",
                "Cheating": "which represents the opposite of fairness, encapsulating instances of unfairness and injustice.",
                "Loyalty": "which deals with group loyalty, self-sacrifice, and vigilance against betrayal",
                "Betrayal":  "which underlies wrongdoing and betraying your group or your relationship.",
                "Authority": "which underlies virtues of leadership and respect within hierarchical relationships",
                "Subversion": "which refers to acts of defiance against authority or hierarchy, and  rebellion against control",
                "Purity": " which is concerned with the sanctity of the body and spirit, promoting virtues like chastity and self-restraint",
                "Degradation": "that denotes the violation of purity and sanctity, including both physical and emotional corruption",
                "Liberty":" which relates to the individual's need to be their own master and to avoid the dominant social mores imposed by the group",
                "Oppression": "that underlies the violation of independence and autonomy, related to unjust treatment."}

In [10]:
all_moral_values = []
for values in moral_values:
  all_moral_values.append(values)
  possible_moral_values = all_moral_values.copy()
all_moral_values = [[value] for value in all_moral_values]

In [11]:
len(possible_moral_values)

12

In [12]:
all_moral_values

[['Care'],
 ['Harm'],
 ['Fairness'],
 ['Cheating'],
 ['Loyalty'],
 ['Betrayal'],
 ['Authority'],
 ['Subversion'],
 ['Purity'],
 ['Degradation'],
 ['Liberty'],
 ['Oppression']]

### OpenAI API Key
Here, insert the API key for OpenAI. You need to create an account and add credit to it in order to use the code below

In [13]:
import openai

openai.api_key = " "

# Utility Functions

#### compute number of tokens and pricing (pricing is per 1000 token)

In [14]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

print(num_tokens_from_string("Hello world, let's test tiktoken.", gpt_version))

9


In [15]:
def compute_actual_price(completion, gpt_version: str) -> int:
    """
    Compute maximum price for API call with given prompt
    The maximum price is computed by assuming the model will output
    the maximum number of tokens that it can output (i.e. 4096).
    Return price in dollars
    """
    price_in = completion.usage.prompt_tokens/1000000*pricings[gpt_version][0]
    price_out = completion.usage.completion_tokens/1000000*pricings[gpt_version][1]

    return price_in+price_out

#### Accessing Moral Values Descriptors

In [16]:
def get_moral_value(moral_value):
  for key, description in moral_values.items():
    if moral_value in key:
      return description

### Vectorise Moral Values
Create vector representation of moral values to store results

In [17]:
from typing import List

def vectorise_moral_values(moral_values: List[int], possible_moral_values):
  moral_values_vector = [0 for _ in range(len(possible_moral_values))]
  for value in moral_values:
    moral_values_vector[value] = 1
  return moral_values_vector

In [18]:
vectorise_moral_values([0, 6], possible_moral_values)

[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

### Prompt Creation and Response Formatting

In [125]:
class JSON_QueryHandler:
    def __init__(self, model, print_output=False):
        self.print_output = print_output
        self.model = model

    def check_response_valid(self, model_response: str):
        if not model_response.strip():
            print("Empty model response.")
            return False, None
        
        valid = True

        try:
            chat_completion_json = json.loads(model_response)
        except json.JSONDecodeError:
            print("Model response: " + str(model_response)[0:50] + "...")
            print("Could not cast model response to JSON. Retrying by trimming ends.")
            try:
                model_response_trimmed = "[" + model_response.split("[", 1)[1]
                model_response_trimmed = model_response_trimmed.split("]", 1)[0] + "]"
                chat_completion_json = json.loads(model_response_trimmed)
            except (json.JSONDecodeError, IndexError) as e:
                print(f"Still could not cast model response to JSON. Error: {str(e)}")
                return False, self.create_cast_to_json_prompt(model_response)

        try:
            moral_values = chat_completion_json["moral_values"]
            for value in moral_values:
                if value not in set(list(range(12))):
                    print(f"Prediction index {value} out of possible moral value keys.")
                    valid = False
        except (TypeError, KeyError) as e:
            print(f"TypeError or KeyError encountered: {str(e)}")
            try:
                for value in chat_completion_json:
                    if value not in set(list(range(12))):
                        print(f"Prediction index {value} out of possible moral value keys.")
                        valid = False
                if valid:
                    chat_completion_json["moral_values"] = chat_completion_json
            except (TypeError, KeyError) as e:
                print(f"TypeError or KeyError encountered: {str(e)}")
                try:
                    moral_values = list(chat_completion_json.values())[0]
                    for value in moral_values:
                        if value not in set(list(range(12))):
                            print(f"Prediction index {value} out of possible moral value keys.")
                            valid = False
                    if valid:
                        chat_completion_json["moral_values"] = moral_values
                except Exception as e:
                    print(f"Exception encountered while processing JSON: {str(e)}")
                    valid = False

        if valid:
            return valid, chat_completion_json
        else:
            return valid, self.create_cast_to_json_prompt(model_response)

    def create_cast_to_json_prompt(self, model_response: str):
        system_template = """
You are a JSON writing assistant. 
When given invalid JSON, you correct it. 
If JSON is already valid, return it as it is.
"""
        system_message_prompt = system_template
        human_prompt = f"Invalid JSON: \n{model_response} \n\nCorrected JSON:\n"
        prompt = [
            {"role": "system", "content": system_message_prompt},
            {"role": "user", "content": human_prompt},
        ]

        return prompt


In [126]:
import re

def prompt_creation(sm_post,
                    gpt_version,
                    option=2,
                    CoT=False):

  if CoT:
    ending = "\n\nLet's think it step by step and output your reasoning together with the json results."
  else:
    ending = "\n\nDO NOT explain your reasoning.\nDO NOT output anything else but the json results."
  if not option:
      system_message = """
                    You will be provided with Social media posts 
                    regarding different social topics. \
                    The Social media posts will be delimited with \
                    #### characters.
                    Classify each social media post into 12 Possible Moral Values \
                    as defined in Moral Foundation Theory.

                    The availablle Moral Values are: \
                    0: care \
                    1: harm \
                    2: fairness \
                    3: cheating \
                    4: loyalty \
                    5: betrayal \
                    6: authority \
                    7: subversion \
                    8: purity \
                    9: degradation\
                    10: liberty\
                    11: oppression

                    Report the results in json format such that the key of the
                    correct moral values is reported in a list:
                    e.g. {"moral_values":[0]}
                    """
  elif option<3:
      system_message = """
                    You will be provided with Social media posts 
                    regarding different social topics. \
                    The Social media posts will be delimited with \
                    #### characters.
                    Classify each social media post into 12 Possible Moral Values \
                    as defined in Moral Foundation Theory.

                    The availablle Moral Values are: \
                    0: care \
                    1: harm \
                    2: fairness \
                    3: cheating \
                    4: loyalty \
                    5: betrayal \
                    6: authority \
                    7: subversion \
                    8: purity \
                    9: degradation\
                    10: liberty\
                    11: oppression

                    This is a multilabel classification problem: more than
                    one category at a time can be assigned.
                    Report the results in json format such that the keys of the
                    correct moral values are reported in a list:
                    e.g. {"moral_values":[0,2]}
                    """
  else:
    system_message = """
                   You will be provided with Social media posts
                    regarding different social topics. \
                    The Social media posts will be delimited with \
                    #### characters.
                    Classify each social media post into 12 Possible Moral Values \
                    as defined in Moral Foundation Theory.

                    The availablle Moral Foundations are: \
                    0: care \
                    1: harm \
                    2: fairness \
                    3: cheating \
                    4: loyalty \
                    5: betrayal \
                    6: authority \
                    7: subversion \
                    8: purity \
                    9: degradation\
                    10: liberty\
                    11: oppression

                    The explanation of the moral foundations is as following:
                    Care: underlies the virtues of kindness, nurturance, empathy, and compassion towards others. 
                    Harm: represents the opposite of care, characterized by violent actions and intentions. 
                    Fairness: underlies virtues of honesty, fair treatment, justice, and dependability. 
                    Cheating: represents the opposite of fairness, encapsulating instances of unfairness and injustice. 
                    Loyalty: represents group loyalty, self-sacrifice, and vigilance against betrayal. 
                    Betrayal: represents the opposite of loyalty and underlies wrongdoing and betraying your group or your relationship. 
                    Authority: underlies virtues of leadership and respect within hierarchical relationships.
                    Subversion: represents the opposite of authority, referring to acts of defiance against hierarchy or tradition and rebellion against control. 
                    Purity: is concerned with the sanctity of the body and spirit, promoting virtues like chastity and self-restraint. 
                    Degradation: represents the opposite of purity, denoting the violation of sanctity, including both physical and emotional corruption.
                    Liberty: is related to the individual's need to be their own master and to avoid the dominant social mores imposed by the group.
                    Oppression: underlies the violation of independence and autonomy, related to unjust treatment.
                    
                    
                    This is a multilabel classification problem: where it's possible to assign one or multiple moral categories simultaneously.
                    It's important to consider the moral foundations and their opposing polarities.
                    (Care versus Harm, Fairness versus Cheating, and Purity versus Degradation, and so on). 
                    Typically, the Social media posts can be interpreted through either a positive (e.g. Loyalty) or negative (e.g. Betrayal) moral foundation, but not both opposites at the same time. 
                    Report the results in json format such that the keys of the
                    correct moral values are reported in a list:
                    e.g. {"moral_values":[0,2]}
                    """

  system_message += ending

  if not option:
    # option 0 does not include any role and it is single label
    prompt = [{"role": "system", "content": system_message},
              {"role": "user", "content": f"####{sm_post}####"}]
  elif option==1:
    # option 1 does not include any role assignment or moral value explanation
    prompt = [{"role": "system", "content": system_message},
              {"role": "user", "content": f"####{sm_post}####"}]
  elif option==2:
    # option 1 does include role assignment but no moral value explanation
    system_message = f"You are an assistant to a researcher studying social psychology. {system_message}"
    prompt = [{"role": "system", "content": system_message},
              {"role": "user", "content": f"####{sm_post}####"}]
  elif option==3:
    # option 2 does not include role assignment. Include moral value explanation
    prompt = [{"role": "system", "content": system_message},
              {"role": "user", "content": f"####{sm_post}####"}]
  elif option==4:
    # option 3 does include role assignment and moral value explanation
    system_message = f"You are an assistant to a researcher studying social psychology. {system_message}"
    prompt = [{"role": "system", "content": system_message},
              {"role": "user", "content": f"####{sm_post}####"}]

  return prompt

def clean_completion(completion):
   return re.sub("\n\(.*?\)\n", "", completion)

def get_completion(sm_post,
                   model,
                   query_handler=None,
                   temperature=0,
                   prompt_option=0,
                   CoT=False):

  prompt = prompt_creation(sm_post, gpt_version=model, option=prompt_option, CoT=CoT)

  response = openai.chat.completions.create(

  model=model,

  messages=prompt,

  temperature=temperature,

  )

  answer = response.choices[0].message.content

  if CoT:
    results = re.findall("\{[\n]?.*?[\n]?\}", answer)
    if 0<len(results)<2:
      json_answer = results[0]
    else:
      actual_price = compute_actual_price(response, model)
      response = "FAILED"
      return response, actual_price, answer
  else:
    json_answer = answer

  actual_price = compute_actual_price(response, model)

  if query_handler is not None:
    valid, response = query_handler.check_response_valid(json_answer)
    if not valid:
      response = openai.chat.completions.create(

                model=model,

                messages=response,

                temperature=temperature,

                )

      actual_price += compute_actual_price(response, model)

      valid, response = query_handler.check_response_valid(response.choices[0].message.content)

      if not valid:
        response = "FAILED"

  return response, actual_price, answer

# Main Process

In [127]:
prompt_option = 4 # use both role assignment and moral value description, define artisti to mimic rather than genre more generally
temperature = 0 # increase for more creative outputs, decrease for more deterministic ones
CoT = True # this option makes the prompt a Chain of Thought, therefore outputting the reasoning of the model together with the answer (and possibly improving results)

In [261]:

import pandas as pd

In [262]:
def save_progress(index, sm_text,results, answers,  filename='gpt_cls_progress.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump((index, sm_text, results, answers), f)

def load_progress(filename='gpt_cls_progress.pkl'):
    try:
        with open(filename, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return [], []


In [263]:
df = test_dataset

### Start runing the GPT model for classification task:

In [2]:
running_price = 0
results = []
answers = []
sm_text = []
index = []
for idx, sm_post in enumerate(df.cleaned_text):
    query_handler = JSON_QueryHandler(gpt_version, print_output=True)
    
    result, price, answer = get_completion(sm_post,
                                         gpt_version,
                                         query_handler,
                                         temperature,
                                         prompt_option,
                                         CoT
                                        )
    running_price += price
    answers.append(answer)
    index.append(idx)
    sm_text.append(df.cleaned_text[idx])  
    try:
        results.append(vectorise_moral_values(result["moral_values"], possible_moral_values))
    except TypeError:
        results.append([0 for _ in range(len(possible_moral_values))])
    if not idx%10:
        print(f"Running price: {running_price}")
        save_progress(index, sm_text, results, answers)

# Save final progress
save_progress(index, sm_text, results, answers)    

In [266]:
result_df = {value:[] for value in possible_moral_values}
result_df["LLM-answer"] = answers

for result in results:
  for is_there, value in zip(result, possible_moral_values):
    result_df[value].append(is_there)

result_df = pd.DataFrame(result_df)

In [ ]:
GPT_human_annotators = pd.concat([df, result_df], axis = 1)

In [297]:
# Save the classification results:
GPT_human_annotators.to_csv('../path/to/saved/csv_file/',
                                          index = None)